1. Engineer your features. Here you do not have them for free. You need to think of possible ways for transforming the collected data into meaningful features. For some ideas, consider traditional features such as texture features, color features, bags of visual words or more powerful ones involving CNNs. If you cannot think of anything, talk to the professor for some ideas.

2. Propose classification techniques to solve the problem. Suggestions here are the CNN directly, or SVMs/Random Forests allied with CNNs through the use of transfer learning.

3. Consider using data augmentation in the training (what about in the testing as well?)

4. Observation: You are free to use any solution to help you extract the features at this point.

5. Report all of your results for the validation and test data. The labels for the test will be released one week before the deadline.

In [6]:
#imports
from glob import glob
from skimage import io
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.optimizers import SGD
from keras.utils import np_utils
import numpy as np
from skimage.transform import resize

In [7]:
#Definicoes
dataFolder = 'data/'
trainFolder = dataFolder + 'train/'
validationFolder = dataFolder + 'val/'
testFolder = dataFolder + 'test/'

numberOfClasses = 83
largura = 200
altura = 200
profundidade = 3

In [8]:
#Classes

class LeNet:
    @staticmethod
    def build(width, height, depth, classes, weightsPath=None):
        # initialize the model
        model = Sequential()
        # first set of CONV => RELU => POOL
        model.add(Convolution2D(20, 5, 5, border_mode="same", input_shape=(height, width, depth)))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), dim_ordering="th"))
        # second set of CONV => RELU => POOL
        model.add(Convolution2D(50, 5, 5, border_mode="same"))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), dim_ordering="th"))
        # set of FC => RELU layers
        model.add(Flatten())
        model.add(Dense(500))
        model.add(Activation("relu"))
 
        # softmax classifier
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        # if a weights path is supplied (inicating that the model was pre-trained), then load the weights
        if weightsPath is not None:
            model.load_weights(weightsPath)
 
        # return the constructed network architecture
        return model

In [10]:
#Funcoes

#Retorna o vetor da imagem dado o nome do seu arquivo
def le_imagem(name):
    return io.imread(name,plugin='matplotlib') 

#Retorna o vetor da imagem e a classe dado o caminho do arquivo
def le_imagem_e_classe(name):
    img = le_imagem(name)
    img = resize(img, (200, 200))
    classe = np_utils.to_categorical(int(name.split('/')[2].split('_')[0]), numberOfClasses)
    return img, classe

#Retorna o caminho de todas as imagens dado a pasta
def nome_das_imagens(pasta):
    return glob(pasta + '*')

#Retorna o numero correspondente a predicao
def categorical_to_number(vector):
    maior = 0
    for i in range(len(vector)):
        if vector[i] > vector[maior]:
            maior = i
    return maior

banana


In [11]:
#Le as imagens de treino
print('lendo imagens de treino')
X_train = []
y_train = []
for file in nome_das_imagens(trainFolder[0:1000]):
    img, classe = le_imagem_e_classe(file)
    X_train.append(img)
    y_train.append(classe)

X_train = np.array(X_train)
y_train = np.array(y_train)
print('lido')

lendo imagens de treino


/usr/local/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


lido


In [12]:
#Le as imagens de validacao
print('lendo imagens de validacao')
X_val = []
y_val = []
for file in nome_das_imagens(validationFolder):
    #img, classe = le_imagem_e_classe(file)
    #X_val.append(img)
    #y_val.append(classe)
    pass

X_val = np.array(X_val)
y_val = np.array(y_val)
print('lido')

lendo imagens de validacao
lido


In [13]:
#Treina essa unica imagem
print('modelando LeNet')
model = LeNet.build(width=largura, height=altura, depth=profundidade, classes=numberOfClasses)
opt = SGD(lr=0.01)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
print('treinando')
model.fit(X_train,y_train)
print('treinado')

modelando LeNet


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(20, (5, 5), input_shape=(200, 200,..., padding="same")`
  if __name__ == '__main__':
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), strides=(2, 2), data_format="channels_first")`
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(50, (5, 5), padding="same")`
  del sys.path[0]
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), strides=(2, 2), data_format="channels_first")`
  from ipykernel import kernelapp as app


treinando
Epoch 1/1
8300/8300 [==============================] - 1831s 221ms/step - loss: 4.4092 - acc: 0.0175
treinado


In [16]:
#Testa o modelo treinado
print('predizendo X_train')
predicted = model.predict(X_train[0:1000])


predizendo X_train


In [17]:
print('Testando os acertos')
acertos = 0.0
for i in range(len(predicted)):
    if categorical_to_number(predicted[i]) == categorical_to_number(y_train[i]):
        acertos += 1.0
print('acertos: ' + str(100.0 * acertos/float(len(predicted))) + '%')


Testando os acertos
acertos: 1.8%
